# Info/CS 4300: Language and Information - in-class demo

# Proto Information Retrieval System

We're going to build a very basic proto-IR system from scratch.  In the first part we will preprocess the "documents" (in this case sentences from a Wikipedia article); in the second part we will implement a method for searching for the set of documents which are closest to a given query.

## Lecture: Basic text processing: sentences, types, tokens, similarity

In [1]:
import re

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
# Copy and pasted background info from the show's Wikipedia page
# (http://en.wikipedia.org/wiki/Keeping_Up_with_the_Kardashians)

background = u"""
Robert Kardashian (1944—2003) and Kristen Mary "Kris" Houghton (born 1955) married in 1978 and had four children together, daughters Kourtney (born 1979), Kim (born 1980), and Khloé, (born 1984), and son Rob (born 1987). The couple divorced in 1991.[3] In 1994, Robert entered the media spotlight when he defended O.J. Simpson for the murders of Nicole Brown Simpson and Ronald Goldman during a lengthy trial. Kris married former Olympic champion Bruce Jenner (born 1949) in 1991. Bruce and Kris had two daughters together, Kendall (born 1995) and Kylie (born 1997). Robert died in 2003 eight weeks after being diagnosed with esophageal cancer.[4] In 2004, Kim became a personal stylist to recording artist Brandy Norwood; she eventually developed into a full-time stylist, and was a personal shopper and stylist to actress Lindsay Lohan.[5] Khloé, Kim and Kourtney further ventured into fashion, opening a high fashion boutique D-A-S-H in Calabasas, California. Throughout Kim's early career, she involved herself in some high-profile relationships including Norwood's brother, rapper Ray J and later, singer Nick Lachey.[5] In 2006, Kourtney starred in her first reality television series, Filthy Rich: Cattle Drive.[6] In February 2007, a home sex video that Kim made with Ray J years earlier was leaked.[7] Vivid Entertainment bought the rights for $1 million and released the film as Kim Kardashian: Superstar on February 21.[7] Kim sued Vivid for ownership of the tape, but dropped the suit in April 2007 and settled with Vivid Entertainment for $5 million.[8]

In August 2007, it was announced that the Kardashians and Jenners would star in a yet-to-be-titled reality show on E!, with Ryan Seacrest serving as executive producer.[6] Seacrest said "At the heart of the series—despite the catfights and endless sarcasm—is a family that truly loves and supports one another [...] The familiar dynamics of this family make them one Hollywood bunch that is sure to entertain." The series announcement came one week after Paris Hilton and her friend Nicole Richie announced that their popular E! series entitled The Simple Life was ending.[6] Keeping Up With the Kardashians premiered on October 14, 2007.[9]

On November 13, 2007, it was announced that E! renewed the show for its second season.[9] The following year, it was renewed for a third season. Lisa Berger, executive vice president of original programming and series development for the network, said "viewers have embraced the Kardashian family and the series has become one of television's most-talked-about shows [...] We are fortunate to work with Seacrest and Bunim-Murray, which have an exceptional ability to capture the Kardashians' hilarious, chaotic and always entertaining personalities and family dynamics."[10] The Hollywood Reporter reported that the family made an estimated $65 million throughout 2010.[3]

In 2011, Kim married NBA player Kris Humphries in a highly publicized wedding ceremony,[11] but filed for divorce 72 days later.[12] This caused widespread backlash from the public and media.[13] Several news outlets surmised that Kardashian's marriage to Humphries was merely a publicity stunt, to promote the Kardashian family's brand and their subsequent television ventures.[14] A widely circulated petition asking to remove all Kardashian-related programming from the air has followed since their split.[15] As of December 2013, eight seasons of the series have aired. On April 24, 2012, E! signed a three-year deal with the Kardashian family that will keep the series airing through seasons seven, eight and nine.[16] The deal was estimated at $40 million.[17][18] On January. 4, 2015, a tenth season was announced to premiere on Feb 8, 2015 after the Kourtney and Khloé Take The Hamptons season finale.

The show revolves around the children of Kris Jenner, and originally focused on her children from her first marriage to deceased attorney Robert Kardashian: Kourtney, Kim, Khloé, and Rob Kourtney's boyfriend Scott Disick is a main character on the show. As the series progressed, Kris' children Kendall and Kylie also became recurring cast members of the show. Kris' second husband[19] 1976 Summer Olympics decathlon champion Bruce Jenner, is also frequently featured on the show, and has been a recurring cast member since the show began.
Since the series' premiere, the Kardashian sisters have established careers in the fashion industry, co-owning the fashion boutique D-A-S-H and launching several fragrances and clothing collections.
Kim gained notoriety as the subject of a sex tape in 2007, and later became involved in a relationship with New Orleans Saints running back Reggie Bush from 2007- March 2010.[20] In 2011, she received widespread criticism after filing for divorce from New Jersey Nets power forward Kris Humphries after a 72-day marriage. In 2012, while still married Kim became pregnant by rapper Kanye West; after suffering from preeclampsia she gave birth prematurely to their daughter North the following June.
Khloé attained notoriety in her own right after being arrested for driving under the influence in 2007, for which she was jailed for approximately three hours in 2008. The following year (during the fourth season), she married Los Angeles Lakers forward Lamar Odom after a one-month relationship. In 2012, she served as a co-host during the second season of the American version of The X Factor.
Rob launched the sock line "Arthur George" in 2012, and was involved in a relationship with singer Adrienne Bailon in the second and third seasons.
Kendall and Kylie have also established careers in the modeling industry.
In the eighth season, Bruce's sons Brandon and Brody Jenner, and Brandon's wife Leah Felder (daughter of Eagles band member Don Felder), were integrated into the supporting cast, while Kourtney, Khloé, and Kim's friends Malika Haqq and Jonathan Cheban joined the series in the second and third seasons.
The family earns an alleged total of $10 million per season of the series.[21]
"""

The simplest way to identify individual words is to split the string at every whitespace. 

In [4]:
background.split()

['Robert',
 'Kardashian',
 '(1944—2003)',
 'and',
 'Kristen',
 'Mary',
 '"Kris"',
 'Houghton',
 '(born',
 '1955)',
 'married',
 'in',
 '1978',
 'and',
 'had',
 'four',
 'children',
 'together,',
 'daughters',
 'Kourtney',
 '(born',
 '1979),',
 'Kim',
 '(born',
 '1980),',
 'and',
 'Khloé,',
 '(born',
 '1984),',
 'and',
 'son',
 'Rob',
 '(born',
 '1987).',
 'The',
 'couple',
 'divorced',
 'in',
 '1991.[3]',
 'In',
 '1994,',
 'Robert',
 'entered',
 'the',
 'media',
 'spotlight',
 'when',
 'he',
 'defended',
 'O.J.',
 'Simpson',
 'for',
 'the',
 'murders',
 'of',
 'Nicole',
 'Brown',
 'Simpson',
 'and',
 'Ronald',
 'Goldman',
 'during',
 'a',
 'lengthy',
 'trial.',
 'Kris',
 'married',
 'former',
 'Olympic',
 'champion',
 'Bruce',
 'Jenner',
 '(born',
 '1949)',
 'in',
 '1991.',
 'Bruce',
 'and',
 'Kris',
 'had',
 'two',
 'daughters',
 'together,',
 'Kendall',
 '(born',
 '1995)',
 'and',
 'Kylie',
 '(born',
 '1997).',
 'Robert',
 'died',
 'in',
 '2003',
 'eight',
 'weeks',
 'after',
 'being

The output is not satisfactory, especially around punctuation. Let's try to catch a higher-level structure first: sentences. A simple idea is to break at periods.

In [5]:
background.split(".")

['\nRobert Kardashian (1944—2003) and Kristen Mary "Kris" Houghton (born 1955) married in 1978 and had four children together, daughters Kourtney (born 1979), Kim (born 1980), and Khloé, (born 1984), and son Rob (born 1987)',
 ' The couple divorced in 1991',
 '[3] In 1994, Robert entered the media spotlight when he defended O',
 'J',
 ' Simpson for the murders of Nicole Brown Simpson and Ronald Goldman during a lengthy trial',
 ' Kris married former Olympic champion Bruce Jenner (born 1949) in 1991',
 ' Bruce and Kris had two daughters together, Kendall (born 1995) and Kylie (born 1997)',
 ' Robert died in 2003 eight weeks after being diagnosed with esophageal cancer',
 '[4] In 2004, Kim became a personal stylist to recording artist Brandy Norwood; she eventually developed into a full-time stylist, and was a personal shopper and stylist to actress Lindsay Lohan',
 '[5] Khloé, Kim and Kourtney further ventured into fashion, opening a high fashion boutique D-A-S-H in Calabasas, Californi

This is not perfect either. One problem consists of the Wikipedia footnotes such as "[9]", which are irrelevant to us. Let's remove them with a regular expression.

In [8]:
re.findall(r"\[\d+\]", background)

['[3]',
 '[4]',
 '[5]',
 '[5]',
 '[6]',
 '[7]',
 '[7]',
 '[8]',
 '[6]',
 '[6]',
 '[9]',
 '[9]',
 '[10]',
 '[3]',
 '[11]',
 '[12]',
 '[13]',
 '[14]',
 '[15]',
 '[16]',
 '[17]',
 '[18]',
 '[19]',
 '[20]',
 '[21]']

In [9]:
background_nofoot = re.sub(r"\[\d+\]", "", background)

We can now try to split sentences again. We also want to split on other punctuation marks, not just the period. This is possible with regular expressions.

In [10]:
splitter = re.compile(r"""
    [.!?]       # split on punctuation
    """, re.VERBOSE)

While this looks better, it messes up som instances: O.J. Simpson's name, the "E!" network's name, and ellipses (...).

To deal with this, we need to think about the decision process of whether a punctuation mark is an end of sentence or not.  In class, we came up with three requirements:

 - It is succeeded by at least one whitespace character (not `O.J`)
 - The first character after the whitespace should be uppercase (not `E! renewed`)
 - The last character before it should not be uppercase (not `J. Simpson`)
 
Are these rules complete?

In [11]:
#Note that we do not need to catch that last period, since we are using split

splitter = re.compile(r"""
    (?<![A-Z])  # last character cannot be uppercase
    [.!?]       # match punctuation
    \s+         # followed by whitespace
    (?=[A-Z])   # next character must be uppercase
    """, re.VERBOSE)

Running our sentence splitter on the text, it seems to do a good job.

In [12]:
for sentence in splitter.split(background_nofoot):
    print(sentence.strip())
    print("--")

Robert Kardashian (1944—2003) and Kristen Mary "Kris" Houghton (born 1955) married in 1978 and had four children together, daughters Kourtney (born 1979), Kim (born 1980), and Khloé, (born 1984), and son Rob (born 1987)
--
The couple divorced in 1991
--
In 1994, Robert entered the media spotlight when he defended O.J. Simpson for the murders of Nicole Brown Simpson and Ronald Goldman during a lengthy trial
--
Kris married former Olympic champion Bruce Jenner (born 1949) in 1991
--
Bruce and Kris had two daughters together, Kendall (born 1995) and Kylie (born 1997)
--
Robert died in 2003 eight weeks after being diagnosed with esophageal cancer
--
In 2004, Kim became a personal stylist to recording artist Brandy Norwood; she eventually developed into a full-time stylist, and was a personal shopper and stylist to actress Lindsay Lohan
--
Khloé, Kim and Kourtney further ventured into fashion, opening a high fashion boutique D-A-S-H in Calabasas, California
--
Throughout Kim's early career,

Now let's identify individual words within each sentence. Rather than splitting at whitespace, let's match all sequences of word characters.

In [13]:
word_splitter = re.compile(r"""
    (\w+)
    """, re.VERBOSE)

In [14]:
sent_words = [word_splitter.findall(sent)
              for sent in splitter.split(background_nofoot)]

In [15]:
sent_words_lower = [[w.lower() for w in sent]
                    for sent in sent_words]

How many sentences do we have?

In [16]:
len(sent_words_lower)

41

In [18]:
sent_words_lower.__class__

list

How many words are there in total? ("tokens")

In [19]:
allwords=[w for sent in sent_words_lower for w in sent]

In [20]:
sorted(allwords)

['1',
 '10',
 '13',
 '14',
 '1944',
 '1949',
 '1955',
 '1976',
 '1978',
 '1979',
 '1980',
 '1984',
 '1987',
 '1991',
 '1991',
 '1994',
 '1995',
 '1997',
 '2003',
 '2003',
 '2004',
 '2006',
 '2007',
 '2007',
 '2007',
 '2007',
 '2007',
 '2007',
 '2007',
 '2007',
 '2008',
 '2010',
 '2010',
 '2011',
 '2011',
 '2012',
 '2012',
 '2012',
 '2012',
 '2013',
 '2015',
 '2015',
 '21',
 '24',
 '4',
 '40',
 '5',
 '65',
 '72',
 '72',
 '8',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'ability',
 'about',
 'actress',
 'adrienne',
 'after',
 'after',
 'after',
 'after',
 'after',
 'after',
 'after',
 'after',
 'air',
 'aired',
 'airing',
 'all',
 'alleged',
 'also',
 'also',
 'also',
 'always',
 'american',
 'an',
 'an',
 'an',
 'and',
 'and',
 'and',
 'and',
 'and',
 'and',
 'and',
 'and',
 'and',
 'and',
 'and',
 'and',
 'and',
 'and',
 'and',
 'and',
 'and',
 'and',
 'and',
 'and',
 'and',
 'and',
 'a

In [0]:
len(allwords)

972

How many distinct types of words ("types") are there?

In [0]:
len(set(allwords))

459

In [0]:
sorted(set(allwords))

['1',
 '10',
 '13',
 '14',
 '1944',
 '1949',
 '1955',
 '1976',
 '1978',
 '1979',
 '1980',
 '1984',
 '1987',
 '1991',
 '1994',
 '1995',
 '1997',
 '2003',
 '2004',
 '2006',
 '2007',
 '2008',
 '2010',
 '2011',
 '2012',
 '2013',
 '2015',
 '21',
 '24',
 '4',
 '40',
 '5',
 '65',
 '72',
 '8',
 'a',
 'ability',
 'about',
 'actress',
 'adrienne',
 'after',
 'air',
 'aired',
 'airing',
 'all',
 'alleged',
 'also',
 'always',
 'american',
 'an',
 'and',
 'angeles',
 'announced',
 'announcement',
 'another',
 'approximately',
 'april',
 'are',
 'around',
 'arrested',
 'arthur',
 'artist',
 'as',
 'asking',
 'at',
 'attained',
 'attorney',
 'august',
 'back',
 'backlash',
 'bailon',
 'band',
 'be',
 'became',
 'become',
 'been',
 'began',
 'being',
 'berger',
 'birth',
 'born',
 'bought',
 'boutique',
 'boyfriend',
 'brand',
 'brandon',
 'brandy',
 'brody',
 'brother',
 'brown',
 'bruce',
 'bunch',
 'bunim',
 'bush',
 'but',
 'by',
 'calabasas',
 'california',
 'came',
 'cancer',
 'capture',
 'care

## Answering queries on the data

We will try to retrieve the closest matching sentence to a given query. To do this, we must define what "closest" means. In other words, we need a similarity measure.

A simple one is the number of types in common between the query and the sentence.

In [21]:
def types_in_common(query_words, sentence):
    A = set(query_words)
    B = set(sentence)
    return len(A.intersection(B))

A slightly more complex one is the the Jaccard similarity measure, which additionaly takes into account the total number of types that the query and the sentence has.

In [22]:
def jaccard(query_words, sentence):
    A = set(query_words)
    B = set(sentence)
    return float(len(A.intersection(B)))/len(A.union(B))

Next we'll define a basic "search engine" which will go through all the sentences and calculate each one's similarity with the query.  It returns a list of sentences sorted by their similarity score (if that score is greater than zero).

To calculate the similarity, this function takes as an argument a similarity_measure function.

In [23]:
from operator import itemgetter

def run_search(query, similarity_measure):
    query_words = word_splitter.findall(query)
    query_words = [w.lower() for w in query_words]
    
    sent_scores = [(sent, similarity_measure(query_words, sent))
                   for sent in sent_words_lower]

    sent_scores = sorted(sent_scores, key=itemgetter(1), reverse=True)
    sent_scores = [(sent, score)
                   for sent, score in sent_scores
                   if score > 0]

    joined_sents = [(" ".join(sent), score) for sent, score in sent_scores]
    return joined_sents

Now we'll run two versions of the search engines (one using the types_in_commmon measure and one using the jaccard measure) for two different queries.

In [24]:
run_search("kris olympic",types_in_common)

[('kris married former olympic champion bruce jenner born 1949 in 1991', 2),
 ('robert kardashian 1944 2003 and kristen mary kris houghton born 1955 married in 1978 and had four children together daughters kourtney born 1979 kim born 1980 and khloé born 1984 and son rob born 1987',
  1),
 ('bruce and kris had two daughters together kendall born 1995 and kylie born 1997',
  1),
 ('in 2011 kim married nba player kris humphries in a highly publicized wedding ceremony but filed for divorce 72 days later',
  1),
 ('the show revolves around the children of kris jenner and originally focused on her children from her first marriage to deceased attorney robert kardashian kourtney kim khloé and rob kourtney s boyfriend scott disick is a main character on the show',
  1),
 ('as the series progressed kris children kendall and kylie also became recurring cast members of the show',
  1),
 ('kris second husband 1976 summer olympics decathlon champion bruce jenner is also frequently featured on the sh

In [25]:
run_search("kourtney",jaccard)

[('in 2006 kourtney starred in her first reality television series filthy rich cattle drive',
  0.07692307692307693),
 ('khloé kim and kourtney further ventured into fashion opening a high fashion boutique d a s h in calabasas california',
  0.05555555555555555),
 ('on january 4 2015 a tenth season was announced to premiere on feb 8 2015 after the kourtney and khloé take the hamptons season finale',
  0.047619047619047616),
 ('robert kardashian 1944 2003 and kristen mary kris houghton born 1955 married in 1978 and had four children together daughters kourtney born 1979 kim born 1980 and khloé born 1984 and son rob born 1987',
  0.03571428571428571),
 ('the show revolves around the children of kris jenner and originally focused on her children from her first marriage to deceased attorney robert kardashian kourtney kim khloé and rob kourtney s boyfriend scott disick is a main character on the show',
  0.030303030303030304),
 ('in the eighth season bruce s sons brandon and brody jenner an

As we will be playing around with the internals of our information retrieval system, let's write a convenience function for displaying the results and highlighting the ones that we know are actually relevant for our information need.

In [26]:
def print_results(orderedlist, relevant_docs=[], maxresults=5):
    """Print search results while highlighting the ones we truly care about"""
    count = 1
    for item, score in orderedlist:
        if item in relevant_docs:
            print("{:d} !!! {:.2f} {}".format(count, score, item))
        elif count <= maxresults:
            print("{:d}     {:.2f} {}".format(count, score, item))
        print(" ")
        count += 1

In [27]:
relevant_docs_champion = ["kris married former olympic champion bruce jenner born 1949 in 1991",
                          "kris second husband 1976 summer olympics decathlon champion bruce jenner is also frequently featured on the show and has been a recurring cast member since the show began"]

The Jaccard measure ranks a completely irrelevant document on #1, gets one right on #2, but the actual best document is all the way down on position 16.

In [28]:
print_results(run_search("the olympic champion in kardashians", jaccard),
              relevant_docs=relevant_docs_champion)

1     0.25 the couple divorced in 1991
 
2 !!! 0.23 kris married former olympic champion bruce jenner born 1949 in 1991
 
3     0.15 keeping up with the kardashians premiered on october 14 2007
 
4     0.14 kendall and kylie have also established careers in the modeling industry
 
5     0.10 in 2012 she served as a co host during the second season of the american version of the x factor
 
 
 
 
 
 
 
 
 
 
 
16 !!! 0.07 kris second husband 1976 summer olympics decathlon champion bruce jenner is also frequently featured on the show and has been a recurring cast member since the show began
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 


#Lecture 5: Vector Space Model

In order to have a more flexible system that we can modify, let's implement a vector space model. We will use term frequency (TF) weights in order to address one of the issues with Jaccard, namely, that if a word occurs more than once, it should naturally matter more.

In [31]:
terms = sorted(set(allwords))

# TF (term frequency) vectorization
# We represent vectors in a "sparse" dictionary format.
# All keys not present in the dictionary are assumed to be zeros.

def doc_to_vec(term_list):
    d = {}
    for v in terms:
        d[v] = term_list.count(v)
    return d

def query_to_vec(term_list):
    d = {}
    for v in terms:
        d[v] = term_list.count(v)
    return d

In [32]:
import math

def dot(d, q):
    sum=0
    for v in d:  # iterates through keys
        sum += d[v] * q[v]
    return sum

One simple similarity measure operating on vectors is the dot product. The higher the dot product between two vectors, the more similar they are.

In [33]:
def dot_measure(query_words, sentence):
    A = query_to_vec(query_words)
    B = doc_to_vec(sentence)
    return float(dot(A, B))

In [34]:
print_results(run_search("the olympic champion in kardashians",dot_measure),relevant_docs=relevant_docs_champion)

1     7.00 lisa berger executive vice president of original programming and series development for the network said viewers have embraced the kardashian family and the series has become one of television s most talked about shows we are fortunate to work with seacrest and bunim murray which have an exceptional ability to capture the kardashians hilarious chaotic and always entertaining personalities and family dynamics the hollywood reporter reported that the family made an estimated 65 million throughout 2010
 
2     6.00 seacrest said at the heart of the series despite the catfights and endless sarcasm is a family that truly loves and supports one another the familiar dynamics of this family make them one hollywood bunch that is sure to entertain the series announcement came one week after paris hilton and her friend nicole richie announced that their popular e series entitled the simple life was ending
 
3     6.00 in the eighth season bruce s sons brandon and brody jenner and brand

We can see that this does even worse, as it rewards longer documents unfairly. We can address that by *length normalizing* the documents and the query, that is, dividing the vectors by their norm.

The resulting measure is the *cosine similarity* measure:

In [35]:
def norm(d):
    sum_sq = 0
    for v in d:
        sum_sq += d[v] * d[v]
    return math.sqrt(sum_sq)

def cos_measure(query_words, sentence):
    A = query_to_vec(query_words)
    B = doc_to_vec(sentence)
    return float(dot(A, B)) / (norm(A) * norm(B))



In [36]:
print_results(run_search("the olympic champion in kardashians",cos_measure),relevant_docs=relevant_docs_champion)

1 !!! 0.40 kris married former olympic champion bruce jenner born 1949 in 1991
 
2     0.40 the couple divorced in 1991
 
3     0.38 rob launched the sock line arthur george in 2012 and was involved in a relationship with singer adrienne bailon in the second and third seasons
 
4     0.34 in 2012 she served as a co host during the second season of the american version of the x factor
 
5     0.33 since the series premiere the kardashian sisters have established careers in the fashion industry co owning the fashion boutique d a s h and launching several fragrances and clothing collections
 
 
 
 
 
 
 
 
 
 
15 !!! 0.24 kris second husband 1976 summer olympics decathlon champion bruce jenner is also frequently featured on the show and has been a recurring cast member since the show began
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 


This already does better, ranking one of our important documents as first. The other is still low in the ranking.

Another issue we have at the moment is that all words matter equally, but intuition dictates that some words in the query (e.g. *olympic*) are more important than others (e.g. *in*).  A way to address this is to weight the words according to their *specificity*, and a concrete implementation is to use *inverse document frequency* (IDF)

In [37]:
IDF = {}
DF = {}

for t in terms:
    DF[t] = len([1 for sent in sent_words_lower if t in sent])
    IDF[t] = 1 / float(DF[t] + 1)

In [38]:
for IDF_t in sorted(IDF.items(), key=itemgetter(1),reverse = False)[:10]:
    print(IDF_t)

print("...")

for IDF_t in sorted(IDF.items(), key=itemgetter(1),reverse = False)[-10:]:
    print(IDF_t)

('the', 0.03225806451612903)
('and', 0.041666666666666664)
('in', 0.043478260869565216)
('a', 0.047619047619047616)
('kim', 0.07692307692307693)
('of', 0.08333333333333333)
('was', 0.08333333333333333)
('for', 0.1)
('series', 0.1)
('to', 0.1)
...
('west', 0.5)
('when', 0.5)
('widely', 0.5)
('wife', 0.5)
('will', 0.5)
('work', 0.5)
('would', 0.5)
('x', 0.5)
('years', 0.5)
('yet', 0.5)


In [39]:
##TF-IDF weights

def doc_to_vec(term_list):
    d = {}
    for v in terms:
        d[v] = term_list.count(v) * IDF[v]
    return d

def query_to_vec(term_list):
    d = {}
    for v in terms:
        d[v] = term_list.count(v) * IDF[v]
    return d

In [40]:
print_results(run_search("the olympic champion in kardashians",cos_measure),relevant_docs=relevant_docs_champion)

1 !!! 0.52 kris married former olympic champion bruce jenner born 1949 in 1991
 
2 !!! 0.10 kris second husband 1976 summer olympics decathlon champion bruce jenner is also frequently featured on the show and has been a recurring cast member since the show began
 
3     0.08 keeping up with the kardashians premiered on october 14 2007
 
4     0.06 in august 2007 it was announced that the kardashians and jenners would star in a yet to be titled reality show on e with ryan seacrest serving as executive producer
 
5     0.03 lisa berger executive vice president of original programming and series development for the network said viewers have embraced the kardashian family and the series has become one of television s most talked about shows we are fortunate to work with seacrest and bunim murray which have an exceptional ability to capture the kardashians hilarious chaotic and always entertaining personalities and family dynamics the hollywood reporter reported that the family made an esti

With TF-IDF and cosine similarity, we now get both relevant documents ranked first and second in the returned list. Neat!

# Lecture- Efficient retrieval

So far we made good progress on improving our IR system along the "ease of access" dimension (basically, getting the most relevant results on top).  Now it's time to tackle about the scalability dimension:  making our IR system answer queries efficiently.

If we time the run of our measure, we see that it takes 40 milliseconds to process a query on our collection of only 41 (short documents).  If the run-time would increases linearly with the number of docs in the collection this would mean that in a web-scale collection of 1 billion documents, it would take us 11 1/2 days to run one search for a query!

***On the tv transcript collection from Assingment 1 (containing about 40,000 sentences) running a single query takes 11 minutes.***

In [41]:
%timeit a=run_search("the olympic champion in kardashians",cos_measure)

18.1 ms ± 737 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Why so slow? let's look at our main data structure: the TERM-DOCUMENT matrix

In [0]:
for d in sent_words_lower:
    print (doc_to_vec(d).values())

dict_values([0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.5, 0.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.16666666666666666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0,

###Building an inverted index
    
The first step is to reorganize the way we store information about which terms appear in which documents.  Instead of storing a huge term-document martrix, we'll use an inverted index: a dictionary where each term links to a list that contains all unique document_ids of the documents in which it appears (a list of "postings").

Normaly this is implemented with linked lists, but for demonstrating the basic concept we will simply use regular lists.

In [42]:
from collections import defaultdict

docs=sent_words_lower

inverted_index=defaultdict(list)
for doc_idx, doc in enumerate(docs): # by iterating this way we make sure that doc_ids will always appear sorted in the postings
    for term in doc:
        if doc_idx not in inverted_index[term]: # this way we have unique doc_ids      
            inverted_index[term].append(doc_idx)  

In [43]:
for term in inverted_index.keys():
    print (term,inverted_index[term])

robert [0, 2, 5, 27]
kardashian [0, 11, 18, 21, 22, 24, 27, 30]
1944 [0]
2003 [0, 5]
and [0, 2, 4, 6, 7, 8, 11, 12, 13, 14, 18, 20, 21, 24, 26, 27, 28, 29, 30, 31, 37, 38, 39]
kristen [0]
mary [0]
kris [0, 3, 4, 19, 27, 28, 29, 32]
houghton [0]
born [0, 3, 4]
1955 [0]
married [0, 3, 19, 33, 35]
in [0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 12, 13, 19, 30, 31, 32, 33, 34, 36, 37, 38, 39]
1978 [0]
had [0, 4]
four [0]
children [0, 27, 28]
together [0, 4]
daughters [0, 4]
kourtney [0, 7, 9, 26, 27, 39]
1979 [0]
kim [0, 6, 7, 8, 10, 11, 12, 19, 27, 31, 33, 39]
1980 [0]
khloé [0, 7, 26, 27, 34, 39]
1984 [0]
son [0]
rob [0, 27, 37]
1987 [0]
the [1, 2, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40]
couple [1]
divorced [1]
1991 [1, 3]
1994 [2]
entered [2]
media [2, 20]
spotlight [2]
when [2]
he [2]
defended [2]
o [2]
j [2, 8, 10]
simpson [2]
for [2, 11, 12, 16, 17, 18, 19, 32, 34]
murders [2]
of [2, 12, 14, 18, 23, 27, 28, 31, 36, 39, 40]
ni

 Note that the document ids are sorted in all postings

### Posting merging algorithm for efficient boolean search

Now we'll see how we can use the inverted index to make an efficient conjunctive boolean search, i.e., search for all documents that contain two given terms.  This can be easily extended to conjunctive searches that span more than two terms, and to more general boolean searches that include disjunctive search.

The return merge posting will contain all the documents that contain both term1 and term2

In [44]:
def merge_postings(term1,term2):
    postings1=inverted_index[term1]
    postings2=inverted_index[term2]
    merged_posting=[]
    i,j=0,0
    while i<len(postings1) and j<len(postings2):
        if postings1[i]==postings2[j]:
            merged_posting.append(postings1[i])
            i+=1
            j+=1
        elif postings1[i]<postings2[j]:
            i+=1
        else:
            j+=1
    return merged_posting


All documents that contain both kim and kris:

In [45]:
merge_postings("kim","kris")

[0, 19, 27]

In [46]:
for doc_idx in merge_postings("kim","kris"):
    print (" ".join(docs[doc_idx]))
    print()

robert kardashian 1944 2003 and kristen mary kris houghton born 1955 married in 1978 and had four children together daughters kourtney born 1979 kim born 1980 and khloé born 1984 and son rob born 1987

in 2011 kim married nba player kris humphries in a highly publicized wedding ceremony but filed for divorce 72 days later

the show revolves around the children of kris jenner and originally focused on her children from her first marriage to deceased attorney robert kardashian kourtney kim khloé and rob kourtney s boyfriend scott disick is a main character on the show



# Efficient ranked retrieval  (using cosine similarity)

But our IR system is a ranked retrieval system using cosine similarity.  Next we'll show how we can use inverted indeces to re-implement it efficiently.

We first need to redefine the inverted indexes to also keep track of term frequencies:


In [54]:
## keep TF together with docs in the postings
docs=sent_words_lower
inverted_index=defaultdict(list)
for doc_idx, doc in enumerate(docs):
    for term in doc: 
# comment out this one, because now I want :
#        if doc_idx not in inverted_index[term]:
            inverted_index[term].append(doc_idx)
    
inverted_index_tf=defaultdict(list)

for term in inverted_index:
    postings=inverted_index[term]
    if len(postings)>0:
        lastdoc=-1
        i=0
        while i<len(postings):
            tf=0
            lastdoc=postings[i]
            while i<len(postings) and lastdoc==postings[i]:
                tf+=1
                lastdoc=postings[i]
                i+=1
            inverted_index_tf[term].append((lastdoc,tf))

        


In [55]:
for term in inverted_index_tf.keys():
    print (term,inverted_index_tf[term])


robert [(0, 1), (2, 1), (5, 1), (27, 1)]
kardashian [(0, 1), (11, 1), (18, 1), (21, 2), (22, 1), (24, 1), (27, 1), (30, 1)]
1944 [(0, 1)]
2003 [(0, 1), (5, 1)]
and [(0, 4), (2, 1), (4, 2), (6, 2), (7, 1), (8, 1), (11, 1), (12, 1), (13, 1), (14, 3), (18, 5), (20, 1), (21, 1), (24, 1), (26, 1), (27, 2), (28, 1), (29, 1), (30, 2), (31, 1), (37, 2), (38, 1), (39, 5)]
kristen [(0, 1)]
mary [(0, 1)]
kris [(0, 1), (3, 1), (4, 1), (19, 1), (27, 1), (28, 1), (29, 1), (32, 1)]
houghton [(0, 1)]
born [(0, 5), (3, 1), (4, 2)]
1955 [(0, 1)]
married [(0, 1), (3, 1), (19, 1), (33, 1), (35, 1)]
in [(0, 1), (1, 1), (2, 1), (3, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 2), (10, 1), (12, 1), (13, 2), (19, 2), (30, 1), (31, 2), (32, 1), (33, 1), (34, 3), (36, 1), (37, 3), (38, 1), (39, 2)]
1978 [(0, 1)]
had [(0, 1), (4, 1)]
four [(0, 1)]
children [(0, 1), (27, 2), (28, 1)]
together [(0, 1), (4, 1)]
daughters [(0, 1), (4, 1)]
kourtney [(0, 1), (7, 1), (9, 1), (26, 1), (27, 2), (39, 1)]
1979 [(0, 1)]
kim [(0,

quick check: we have 3 documents containing "born", with 5, 1 and 2 ocurances of the term:

In [56]:
inverted_index_tf["born"]

[(0, 5), (3, 1), (4, 2)]

In [57]:
docs[0].count("born")

5

In [58]:
" ".join(docs[0])

'robert kardashian 1944 2003 and kristen mary kris houghton born 1955 married in 1978 and had four children together daughters kourtney born 1979 kim born 1980 and khloé born 1984 and son rob born 1987'

### Cosine similarity with inverted index (with some shortcuts)

Now we define a method that searches for a query efficiently using the inverted index.  Our goal is to get the same results as before, but much more efficiently.  The main intuition is to use the indexes to go only through the douments that have a chance to be relevant.

Note that some of the implementation details are inefficient (efficient implementations are part of your assigment)

In [64]:
inverted_index_tf

defaultdict(list,
            {'robert': [(0, 1), (2, 1), (5, 1), (27, 1)],
             'kardashian': [(0, 1),
              (11, 1),
              (18, 1),
              (21, 2),
              (22, 1),
              (24, 1),
              (27, 1),
              (30, 1)],
             '1944': [(0, 1)],
             '2003': [(0, 1), (5, 1)],
             'and': [(0, 4),
              (2, 1),
              (4, 2),
              (6, 2),
              (7, 1),
              (8, 1),
              (11, 1),
              (12, 1),
              (13, 1),
              (14, 3),
              (18, 5),
              (20, 1),
              (21, 1),
              (24, 1),
              (26, 1),
              (27, 2),
              (28, 1),
              (29, 1),
              (30, 2),
              (31, 1),
              (37, 2),
              (38, 1),
              (39, 5)],
             'kristen': [(0, 1)],
             'mary': [(0, 1)],
             'kris': [(0, 1),
              (3, 1),
        

In [65]:
## let's use the TF-IDF weighting
def docweight(queryterm, tf):
    return tf*IDF[queryterm]

doc_norms={}
for doc_idx,doc in enumerate(docs):
    doc_norms[doc_idx]=norm(doc_to_vec(doc))  ## this can be done more efficiently, but I only do it once, so i don't care 

    
def run_search_on_index(query):
    query_words = word_splitter.findall(query)  #1
    query_words = [w.lower() for w in query_words]  #2
    query_weights_dict={} #3  ## precomputed nonzero query  weights
    large_vector=query_to_vec(query_words) #4
    for t in large_vector: #5
        if large_vector[t]>0: #6
            query_weights_dict[t]=large_vector[t] #7

    ## these are the score accumulators
    ## they will accumulate accumulate TF-IDF weight products (w_iq * w_ij) 
    doc_scores=defaultdict(int)  
    
    ## since keys of our index are query words, we will iterate through those
    for queryterm in query_words: #8
        postings=inverted_index_tf[queryterm] #9
        for doc_idx,tf in postings:  #10 ## only touching docs that are involved in the query
            weight_update=query_weights_dict[queryterm]*docweight(queryterm,tf) #11
            doc_scores[doc_idx]+=weight_update #12 ## accumulate TF-IDF updates here 
            ##(at the end of the outer loop, for each document that has any of these terms
            ##  we will have accumulated all TF-IDF weights that are needed to compute the 
            ## dot-product part of the cosine similarity score

    for d in doc_scores: #13
        doc_scores[d]=doc_scores[d]/(doc_norms[d]*norm(query_weights_dict)) #14 
    
    doc_idx_scores = sorted(doc_scores.items(), key=itemgetter(1), reverse=True) 
    doc_scores = [(docs[doc_idx], score)
                   for doc_idx, score in doc_idx_scores
                   if score > 0]

    joined_sents = [(" ".join(sent), score) for sent, score in doc_scores]
    return joined_sents

*** Quiz (submit answers on CMS): Change the query such that you get the same order of results but faster time**

In [66]:
original_query = "the olympic champion in kardashians"
modified_query = "FILL IN"

print_results(run_search_on_index(modified_query),relevant_docs=relevant_docs_champion)

1     0.09 rob launched the sock line arthur george in 2012 and was involved in a relationship with singer adrienne bailon in the second and third seasons
 
2     0.07 khloé attained notoriety in her own right after being arrested for driving under the influence in 2007 for which she was jailed for approximately three hours in 2008
 
3     0.06 in 2006 kourtney starred in her first reality television series filthy rich cattle drive
 
4     0.06 the couple divorced in 1991
 
5     0.05 in 2011 kim married nba player kris humphries in a highly publicized wedding ceremony but filed for divorce 72 days later
 
 
 
 
9 !!! 0.04 kris married former olympic champion bruce jenner born 1949 in 1991
 
 
 
 
 
 
 
 
 
 
 
 
 
 


In [67]:
## Paste the result of this line
%timeit a=run_search_on_index(modified_query)

143 µs ± 1 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [68]:
## for your own sake, compare with the speedup when using term-document matrix implementation
%timeit a=run_search(modified_query,cos_measure)

15.7 ms ± 389 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


**Optional participation question: Add one line to the code below to apply the same speed-up intuition more generally (do not rely on specific words)**

(The extra line needs to be added in between two of the numbered lines e.g., #1,#2,#3..#7.)

In [0]:
def run_search_on_index(query):
    query_words = word_splitter.findall(query) 
    query_words = [w.lower() for w in query_words] 
    query_weights_dict={}   
    large_vector=query_to_vec(query_words)
    for t in large_vector: 
        if large_vector[t]>0:
            query_weights_dict[t]=large_vector[t]

    ## these are the score accumulators
    ## they will accumulate accumulate TF-IDF weight products (w_iq * w_ij) 
    doc_scores=defaultdict(int)  
    
    ## since keys of our index are query words, we will iterate through those
    for queryterm in query_words: #1
      
        postings=inverted_index_tf[queryterm] #2
        for doc_idx,tf in postings:  #3 
            weight_update=query_weights_dict[queryterm]*docweight(queryterm,tf) #4
            doc_scores[doc_idx]+=weight_update #5 

    for d in doc_scores: #6
        doc_scores[d]=doc_scores[d]/(doc_norms[d]*norm(query_weights_dict)) #7 
    
    doc_idx_scores = sorted(doc_scores.items(), key=itemgetter(1), reverse=True) 
    doc_scores = [(docs[doc_idx], score)
                   for doc_idx, score in doc_idx_scores
                   if score > 0]

    joined_sents = [(" ".join(sent), score) for sent, score in doc_scores]
    return joined_sents

In [0]:
print_results(run_search_on_index("sock line"),relevant_docs=relevant_docs_champion)

NameError: ignored

In [0]:
## Paste the result of this line
%timeit a=run_search_on_index(original_query)